# Segmenting and Clustering Neighborhoods in Toronoto

## Part 1
**Objectives:** 
1) Web scrape Toronto's neighborhood data from a Wikipedia page using BeautifulSoup

2) Add data to pandas dataframe 

In [1]:
#install libraries
#!pip install pandas
#!pip install bs4
#!pip install requests

#import import libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
#assign url of the Wikipedia Page to url variable
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
#download the webpage
html_data=requests.get(url).text
#parse the page using BeautifulSoup
soup =BeautifulSoup(html_data, 'html5lib')

In [3]:
#create a blank list
table_contents=[]
#use a for loop to append information to the list
for row in soup.find('table').findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

In [4]:
#load dictionary into a data frame
df=pd.DataFrame(table_contents)
#rename Boroughs
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [5]:
df.shape

(103, 3)

## Part 2
**Objectives:**
1) Assign latitude and longitude to each postal code in Toronto

In [6]:
#read the csv file instead of geocoder
df2=pd.read_csv('Geospatial_Coordinates.csv')
#rename columns before merging two dataframes
df2.columns=['PostalCode','Latitude', 'Longitude']

In [7]:
#merge dataframes
toronto_data=pd.merge(df,df2, on='PostalCode',how='inner')

## Part 3
**Objectives:**
1) Utilize the Foursquare location data to explore and cluster the neighborhoods in Tornoto

2) Generate maps to visualize neighborhoods and how they cluster together

**Install/Import Required Libraries** 

In [8]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim
import numpy as np
# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

Solving environment: done

# All requested packages already installed.



**Define Foursquare Credentials and Version**

In [9]:
CLIENT_ID = 'LQQFD2FGNWDWUB3OXGP4YC4JHPR1VO3X0DP1KRREVEKJ4MBV' #Foursquare ID
CLIENT_SECRET = '2OC43CCJFC1XDWW0QG4VQUQR4NXOBTEMHVYKV1V2DOBCRTI5' #Foursquare Secret
ACCESS_TOKEN = 'IUNOUPL1EBYLVUXHTVOPF4VZX50DSWZ1530O4JQNG1MAKARC' #FourSquare Access Token
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LQQFD2FGNWDWUB3OXGP4YC4JHPR1VO3X0DP1KRREVEKJ4MBV
CLIENT_SECRET:2OC43CCJFC1XDWW0QG4VQUQR4NXOBTEMHVYKV1V2DOBCRTI5


**Define coordinates for Toronto** 

In [10]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="rpk_coursera")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(location.latitude,location.longitude)

43.6534817 -79.3839347


**Focus on Boroughs that Include "Toronto" in their Name**

In [11]:
toronto_data=toronto_data[toronto_data['Borough'].str.contains('Toronto')]
toronto_data.reset_index(drop=True, inplace=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


**Create map of Toronto using latitude and longitude values defined above**

In [12]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

**Define a function to collect venue information and download it**

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

**Explore the number of total venues and how many venues there are per neighborhood**

In [15]:
print(toronto_venues.shape)
toronto_venues.groupby('Neighborhood').count()

(1583, 7)


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,59,59,59,59,59,59
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,64,64,64,64,64,64
Christie,15,15,15,15,15,15
Church and Wellesley,78,78,78,78,78,78
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,37,37,37,37,37,37
Davisville North,8,8,8,8,8,8


In [16]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
#assign neighborhoods to the column neighborhood
#reorder columns
columnorder=['Neighborhood','Adult Boutique', 'Airport', 'Airport Food Court', 'Airport Gate', 'Airport Lounge', 'Airport Service', 'Airport Terminal', 'American Restaurant', 'Antique Shop', 'Aquarium', 'Art Gallery', 'Art Museum', 'Arts & Crafts Store', 'Asian Restaurant', 'Athletics & Sports', 'Auto Workshop', 'BBQ Joint', 'Baby Store', 'Bagel Shop', 'Bakery', 'Bank', 'Bar', 'Baseball Stadium', 'Basketball Stadium', 'Beach', 'Bed & Breakfast', 'Beer Bar', 'Beer Store', 'Belgian Restaurant', 'Bistro', 'Boat or Ferry', 'Bookstore', 'Boutique', 'Brazilian Restaurant', 'Breakfast Spot', 'Brewery', 'Bubble Tea Shop', 'Burger Joint', 'Burrito Place', 'Bus Line', 'Business Service', 'Butcher', 'Café', 'Cajun / Creole Restaurant', 'Candy Store', 'Caribbean Restaurant', 'Cheese Shop', 'Chinese Restaurant', 'Chocolate Shop', 'Church', 'Climbing Gym', 'Clothing Store', 'Cocktail Bar', 'Coffee Shop', 'College Arts Building', 'College Gym', 'College Rec Center', 'Colombian Restaurant', 'Comfort Food Restaurant', 'Comic Shop', 'Concert Hall', 'Convenience Store', 'Cosmetics Shop', 'Coworking Space', 'Creperie', 'Cuban Restaurant', 'Cupcake Shop', 'Dance Studio', 'Deli / Bodega', 'Department Store', 'Dessert Shop', 'Dim Sum Restaurant', 'Diner', 'Discount Store', 'Distribution Center', 'Dog Run', 'Doner Restaurant', 'Donut Shop', 'Dumpling Restaurant', 'Eastern European Restaurant', 'Electronics Store', 'Escape Room', 'Ethiopian Restaurant', 'Event Space', 'Falafel Restaurant', 'Farmers Market', 'Fast Food Restaurant', 'Filipino Restaurant', 'Fish & Chips Shop', 'Fish Market', 'Flea Market', 'Flower Shop', 'Food & Drink Shop', 'Food Court', 'Food Truck', 'Fountain', 'French Restaurant', 'Fried Chicken Joint', 'Frozen Yogurt Shop', 'Fruit & Vegetable Store', 'Furniture / Home Store', 'Gaming Cafe', 'Garden', 'Garden Center', 'Gas Station', 'Gastropub', 'Gay Bar', 'General Entertainment', 'General Travel', 'German Restaurant', 'Gift Shop', 'Gluten-free Restaurant', 'Gourmet Shop', 'Greek Restaurant', 'Grocery Store', 'Gym', 'Gym / Fitness Center', 'Harbor / Marina', 'Health & Beauty Service', 'Health Food Store', 'Historic Site', 'History Museum', 'Hookah Bar', 'Hospital', 'Hotel', 'Hotel Bar', 'IT Services', 'Ice Cream Shop', 'Indian Restaurant', 'Indie Movie Theater', 'Intersection', 'Irish Pub', 'Italian Restaurant', 'Japanese Restaurant', 'Jazz Club', 'Jewelry Store', 'Juice Bar', 'Korean Restaurant', 'Lake', 'Latin American Restaurant', 'Light Rail Station', 'Lingerie Store', 'Liquor Store', 'Lounge', 'Market', 'Martial Arts School', 'Mediterranean Restaurant', "Men's Store", 'Metro Station', 'Mexican Restaurant', 'Middle Eastern Restaurant', 'Miscellaneous Shop', 'Modern European Restaurant', 'Molecular Gastronomy Restaurant', 'Monument / Landmark', 'Moroccan Restaurant', 'Movie Theater', 'Museum', 'Music Venue', 'New American Restaurant', 'Nightclub', 'Noodle House', 'Office', 'Opera House', 'Optical Shop', 'Organic Grocery', 'Other Great Outdoors', 'Park', 'Performing Arts Venue', 'Pet Store', 'Pharmacy', 'Pizza Place', 'Plane', 'Playground', 'Plaza', 'Poke Place', 'Portuguese Restaurant', 'Poutine Place', 'Pub', 'Ramen Restaurant', 'Record Shop', 'Rental Car Location', 'Restaurant', 'Roof Deck', 'Sake Bar', 'Salad Place', 'Salon / Barbershop', 'Sandwich Place', 'Scenic Lookout', 'Sculpture Garden', 'Seafood Restaurant', 'Shoe Repair', 'Shoe Store', 'Shopping Mall', 'Skate Park', 'Skating Rink', 'Smoke Shop', 'Smoothie Shop', 'Snack Place', 'Soup Place', 'Spa', 'Speakeasy', 'Sporting Goods Shop', 'Sports Bar', 'Stadium', 'Stationery Store', 'Steakhouse', 'Strip Club', 'Supermarket', 'Sushi Restaurant', 'Swim School', 'Taco Place', 'Tailor Shop', 'Taiwanese Restaurant', 'Tanning Salon', 'Tea Room', 'Tennis Court', 'Thai Restaurant', 'Theater', 'Theme Restaurant', 'Tibetan Restaurant', 'Toy / Game Store', 'Trail', 'Train Station', 'Vegetarian / Vegan Restaurant', 'Video Game Store', 'Vietnamese Restaurant', 'Wine Bar', 'Wine Shop', 'Yoga Studio']
toronto_onehot=toronto_onehot[columnorder]
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
toronto_onehot.head()

,Neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
toronto_onehot.shape

(1583, 231)

**Group Venue Data by Neighborhood**

In [18]:
toronto_grouped=toronto_onehot.groupby("Neighborhood").mean().reset_index()
toronto_grouped.head()

,Neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.016949,0.0,0.0,0.000000,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.058824,0.058824,0.058824,0.117647,0.117647,0.117647,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
3,Central Bay Street,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.015625,0.0,0.0,0.015625,0.0,0.015625
4,Christie,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000


**Explore Top 5 Venues for a given Neighborhood**

In [19]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.10
1  Cocktail Bar  0.07
2        Bakery  0.05
3    Restaurant  0.03
4   Cheese Shop  0.03


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.14
1  Breakfast Spot  0.09
2     Coffee Shop  0.09
3             Gym  0.05
4      Restaurant  0.05


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3          Boutique  0.06
4           Airport  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.19
1      Sandwich Place  0.06
2                Café  0.05
3  Italian Restaurant  0.05
4        Burger Joint  0.03


----Christie----
           venue  freq
0  Grocery Store  0.27
1           Café  0.20
2           Park  0.13
3    Coffee Shop  0.07
4    Candy Store  0.07


----Church and Welle

**Define a function for returning most common neighborhood**

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Beer Bar,Cheese Shop,Pharmacy,Farmers Market,Restaurant,Seafood Restaurant,Café
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Grocery Store,Bakery,Performing Arts Venue,Pet Store,Nightclub,Climbing Gym,Restaurant
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Boutique,Rental Car Location,Coffee Shop,Plane,Boat or Ferry,Bar
3,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Salad Place,Burger Joint,Bubble Tea Shop,Business Service,Restaurant,Portuguese Restaurant
4,Christie,Grocery Store,Café,Park,Baby Store,Candy Store,Coffee Shop,Nightclub,Restaurant,Italian Restaurant,Eastern European Restaurant


**Run k-means to cluster the neighborhood into 5 clusters.**

In [22]:
# set number of clusters
kclusters = 8

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 0, 3, 3, 0, 3, 3, 3, 3, 3], dtype=int32)

In [26]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ValueError: cannot insert Cluster Labels, already exists

In [27]:
toronto_merged = toronto_data

In [28]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [29]:
toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,3,Coffee Shop,Café,Park,Pub,Bakery,Breakfast Spot,Theater,Brewery,Spa,Beer Store
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,3,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,Italian Restaurant,Bubble Tea Shop,Fast Food Restaurant,Ramen Restaurant
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,3,Coffee Shop,Café,Gastropub,Beer Bar,Cocktail Bar,Cosmetics Shop,Hotel,Clothing Store,Lingerie Store,Restaurant
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,7,Pub,Coffee Shop,Health Food Store,Trail,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,3,Coffee Shop,Cocktail Bar,Bakery,Beer Bar,Cheese Shop,Pharmacy,Farmers Market,Restaurant,Seafood Restaurant,Café


In [30]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters